In [1]:
import os
from os.path import isdir, join
from pathlib import Path
import pandas as pd

import tensorflow as tf
# Math
import numpy as np
from scipy.fftpack import fft
from scipy import signal
from scipy.io import wavfile
import librosa

from sklearn.decomposition import PCA

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
import librosa.display

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import pandas as pd

%matplotlib inline

In [2]:
import glob
uav_path = '../data/44100/Unloaded/*.*'
loaded_path = '../data/44100/Loaded/*.*'
none_path = '../data/44100/Background/*.*'

uav_files = glob.glob(uav_path)
loaded_files = glob.glob(loaded_path)
none_files = glob.glob(none_path)

In [3]:
print(len(uav_files),'개\t', uav_files[0])
print(len(uav_files),'개\t', loaded_files[0])
print(len(none_files), '개\t',none_files[0])

10 개	 ../data/44100/Unloaded\P1_stationary.wav
10 개	 ../data/44100/Loaded\test_1532717717-loaded.wav
50 개	 ../data/44100/Background\background_06_02_01.wav


# Load Data

The reason of why SR is 44100 is that the sample rate of above files is 44.1kbps

a wav file sample has 884736. if sample is divided by sample rate, the value is time
the time is fixed by 20.06

In [4]:
def load(files, sr=44100):
    [raw, sr] = librosa.load(files[0], sr=sr)
    for f in files[1:]:
        [array, sr] = librosa.load(f, sr=sr)
        raw = np.hstack((raw, array))
    print(raw.shape)
    return raw

In [5]:
uav_raw = load(uav_files)
loaded_raw = load(loaded_files)
none_raw = load(none_files)

(34496499,)
(16942106,)
(81385951,)


# Feature extraction 
## steps
#### 1. Resampling 
#### 2. *VAD*( Voice Activity Detection)
#### 3. Maybe padding with 0 to make signals be equal length
#### 4. Log spectrogram (or *MFCC*, or *PLP*)
#### 5. Features normalization with *mean* and *std*
#### 6. Stacking of a given number of frames to get temporal information



## 1. Resampling

if you see the graph, there are few at high frequency. this is mean that data is big but it's no useless. so To small the data, do Resampling. In general, use 0~8000Hz 

## 2. VAD

Sometimes, Files have silence. It is not necessary. So, We need to find sound of Drone except silence.

But, Not yet implemented

## 3. padding with 0 to make signals be equal length

If we have a lot of sound files, we need to pad some datas. But These files's time is longger than 1 second. So It dosn't need to pad

## 4. Log spectrogram (or MFCC, or PLP)

The upper picture is resampled data. 
The lower picture is original data.

In MFCC Feature, There is no big difference. 

In [6]:
from scipy.stats import skew
#returns mfcc features with mean and standard deviation along time
N_MFCC = 20
def mfcc4(raw, label, chunk_size=8192, window_size=4096, sr=44100, n_mfcc=16, n_frame=16):
    mfcc = np.empty((0, n_mfcc* n_frame))
    y = []
    print(raw.shape)
    for i in range(0, len(raw), chunk_size//2):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc) #n_mfcc,17
        if mfcc_slice.shape[1] < 17:
            print("small end:", mfcc_slice.shape)
            continue
        mfcc_slice = mfcc_slice[:,:-1]
        #print(mfcc_slice.shape)
        mfcc_slice = mfcc_slice.reshape((1, mfcc_slice.shape[0]* mfcc_slice.shape[1]))
        #print(mfcc_slice.shape)
        mfcc = np.vstack((mfcc, mfcc_slice))
        y.append(label)
    y = np.array(y)
    return mfcc, y

def log_specgram(audio, sample_rate, window_size=20,
                 step_size=10, eps=1e-10):
    nperseg = int(round(window_size * sample_rate / 1e3))
    noverlap = int(round(step_size * sample_rate / 1e3))
    freqs, times, spec = signal.spectrogram(audio,
                                    fs=sample_rate,
                                    window='hann',
                                    nperseg=nperseg,
                                    noverlap=noverlap,
                                    detrend=False)
    return freqs, times, np.log(spec.T.astype(np.float32) + eps)
def mfcc(raw, chunk_size=8192, sr=44100, n_mfcc=N_MFCC):
    mfcc = np.empty((N_MFCC, 0))
    for i in range(0, len(raw), chunk_size):
        mfcc_slice = librosa.feature.mfcc(raw[i:i+chunk_size], sr=sr, n_mfcc=n_mfcc)
        mfcc = np.hstack((mfcc, mfcc_slice))
    return mfcc

In [7]:
'''
mfcc_uav, y_uav = mfcc4(uav_raw, 1)
print(mfcc_uav.shape, y_uav.shape)
mfcc_none, y_none = mfcc4(none_raw, 0)
print(mfcc_none.shape, y_none.shape)
'''
mfcc_uav = mfcc(uav_raw)
mfcc_loaded = mfcc(loaded_raw)
mfcc_none = mfcc(none_raw)
print(len(mfcc_uav),len(mfcc_loaded),len(mfcc_none) )

20 20 20


## 5. Features normalization with *mean* and *std*

## 6. Stacking of a given number of frames to get temporal information

In [8]:
# or should we give one label to one chunk?
y_uav = np.ones(mfcc_uav.shape[1], dtype=int)*2
y_loaded = np.ones(mfcc_loaded.shape[1], dtype=int)
y_none =np.zeros(mfcc_none.shape[1], dtype=int)

print(y_uav.shape, y_uav[0])
print(y_loaded.shape, y_loaded[0])
print(y_none.shape, y_none[0])

X = np.hstack((mfcc_uav, mfcc_loaded))
X = np.hstack((X, mfcc_none)).T

y = np.hstack((y_uav, y_loaded))
y = np.hstack((y, y_none))
print(X.shape, y.shape)

(71586,) 2
(35159,) 1
(168891,) 0
(275636, 20) (275636,)


In [9]:
n_labels = y.shape[0]
n_unique_labels = 3
y_encoded = np.zeros((n_labels, n_unique_labels))
y_encoded[np.arange(n_labels), y] = 1
print(y_encoded.shape)
print(y_encoded[0], y_encoded[40000],y_encoded[100000])

(275636, 3)
[0. 0. 1.] [0. 0. 1.] [0. 1. 0.]


In [10]:
'''
mfcc_uav_list = mfcc_uav.tolist()
mfcc_uav_list = mfcc_uav_list
fig = plt.figure(figsize=(15,9))
ax = fig.add_subplot(1,1,1)
ax.plot(np.linspace(0,len(mfcc_uav_list), len(mfcc_uav_list)),mfcc_uav_list)
'''

'\nmfcc_uav_list = mfcc_uav.tolist()\nmfcc_uav_list = mfcc_uav_list\nfig = plt.figure(figsize=(15,9))\nax = fig.add_subplot(1,1,1)\nax.plot(np.linspace(0,len(mfcc_uav_list), len(mfcc_uav_list)),mfcc_uav_list)\n'

In [11]:
dataX = X
dataY = y_encoded
print(y_encoded)
print(dataX.shape, dataY.shape)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
(275636, 20) (275636, 3)


In [12]:
def makeHot(dataX, dataY, seq_length):
    X_hot_list= []
    Y_hot_tmp = dataY[seq_length-1:]

    for i in range(0, dataX.shape[0] - seq_length+1):
        _x = dataX[i:i + seq_length]
        #if i<10:
            #print(_x, "->", Y_hot_tmp[i])
        X_hot_list.append(_x)

    X_hot = np.array(X_hot_list[:])
    Y_hot= Y_hot_tmp.reshape((len(Y_hot_tmp),n_unique_labels))
    return X_hot[:], Y_hot[:]

In [13]:
seq_length = 16 #layer
X_hot, Y_hot = makeHot(dataX, dataY, seq_length)
print(X_hot.shape, Y_hot.shape)


(275621, 16, 20) (275621, 3)


In [14]:
class Data:
    def __init__(self,X,Y,BatchSize):
        self.X = X
        self.Y = Y
        self.len = len(Y)
        self.bs = BatchSize
        self.bs_i = 0
    def getBatchData(self):
        s = self.bs_i
        e = self.bs_i + self.bs
        if e> self.len:
            e -= self.len
            result =  np.vstack((self.X[s:],self.X[:e])), np.vstack((self.Y[s:],self.Y[:e]))
        else:
            result =  self.X[s:e], self.Y[s:e]
            
        self.bs_i = e
        return result
        


In [15]:
dataX = [1,2,3,4,5,6,7,8]
dataY = [11,12,13,14,15,16,17,18]
D = Data(dataX, dataY,3)
x, y = D.getBatchData()
print(x,y)
x, y = D.getBatchData()
print(x,y)

[1, 2, 3] [11, 12, 13]
[4, 5, 6] [14, 15, 16]


In [16]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_hot, Y_hot, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [17]:
batch_size = 2048
traindata = Data(X_train,y_train,batch_size)
testdata = Data(X_test,y_test,batch_size)
valdata = Data(X_val,y_val,batch_size)

In [18]:
print(X_train.shape, X_test.shape,X_val.shape)
print(y_train.shape, y_test.shape,y_val.shape)

(176396, 16, 20) (55125, 16, 20) (44100, 16, 20)
(176396, 3) (55125, 3) (44100, 3)


In [19]:
'''
np.save('../data/Xy/X_train2', X_train)
np.save('../data/Xy/X_test2', X_test)
np.save('../data/Xy/y_train2', y_train)
np.save('../data/Xy/y_test2', y_test)
'''

"\nnp.save('../data/Xy/X_train2', X_train)\nnp.save('../data/Xy/X_test2', X_test)\nnp.save('../data/Xy/y_train2', y_train)\nnp.save('../data/Xy/y_test2', y_test)\n"

In [20]:
'''
X_train = np.load('../data/Xy/X_train2.npy')
X_test = np.load('../data/Xy/X_test2.npy')
y_train = np.load('../data/Xy/y_train2.npy')
y_test = np.load('../data/Xy/y_test2.npy')
'''

"\nX_train = np.load('../data/Xy/X_train2.npy')\nX_test = np.load('../data/Xy/X_test2.npy')\ny_train = np.load('../data/Xy/y_train2.npy')\ny_test = np.load('../data/Xy/y_test2.npy')\n"

# Tensorflow RNN

## Train 

In [21]:
batch_size = batch_size
num_classes = N_MFCC            #분류할 사전의 크기 

learning_rate = 0.01
sequence_length = seq_length #9         

output_dim = n_unique_labels
layers = 3

In [22]:
X = tf.placeholder(tf.float32, [None, sequence_length,num_classes], name="X")
Y = tf.placeholder(tf.float32, [None, output_dim], name="Y")

cell = tf.contrib.rnn.BasicLSTMCell(num_units=num_classes, state_is_tuple=True)
cell = tf.contrib.rnn.MultiRNNCell([cell]*layers, state_is_tuple= True)

BatchSize = tf.placeholder(tf.int32, [], name='BatchSize')
initial_state = cell.zero_state(BatchSize, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X,initial_state=initial_state,dtype=tf.float32)

dense1 = tf.contrib.layers.fully_connected(outputs[:,-1], output_dim, activation_fn=None)
dense2 = tf.layers.dense(inputs=dense1, units=num_classes, activation=tf.nn.relu)

Y_pred= tf.layers.dense(inputs=dense2, units=output_dim)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Y_pred, labels=Y))
lr = tf.placeholder(tf.float32,shape=(), name='learning_rate')
train = tf.train.AdamOptimizer(lr).minimize(cost)


In [23]:
traindata.X[0]

array([[-5.24285461e+02,  1.05689688e+02,  2.34859049e+01,
         1.56318004e+01,  5.53109457e+00,  8.71770189e+00,
         6.21920751e+00, -9.77620163e-01,  6.41941339e+00,
         6.89265910e+00, -3.88062747e+00, -2.30095251e+00,
         1.71102388e-01,  2.29457925e+00,  9.54123633e+00,
         5.03520078e+00,  7.76846924e-01,  3.28094293e+00,
         8.47739353e-01,  3.16145109e+00],
       [-5.32972639e+02,  1.04444533e+02,  2.51869709e+01,
         1.30460006e+01,  4.81677191e+00,  9.46346030e+00,
         1.05919269e+01,  4.85616563e+00,  6.22374116e+00,
         3.19416144e+00, -6.73717896e+00, -4.05999290e+00,
         1.63683914e+00,  1.09187574e+00,  6.84105776e+00,
         4.25543155e+00, -2.68917616e+00,  1.43046370e+00,
        -2.27906680e+00,  4.28891601e+00],
       [-5.26164647e+02,  1.09428217e+02,  2.24643161e+01,
         1.20741965e+01,  2.50434022e-01,  8.15456427e+00,
         9.48548403e+00,  2.20058872e+00,  4.22555358e+00,
         4.21675033e+00, -5.3

In [24]:
print(traindata.Y)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [25]:
x, y = traindata.getBatchData()
print(y)

[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [ ]:
from sklearn.metrics import accuracy_score
init = tf.global_variables_initializer()
cost_history = np.empty(shape=[1],dtype=float)
step_loss = 999999.0
model_path = '../models/RNN/my_RNN_model'
saver = tf.train.Saver()
training_epochs = 200
# Training step

sess = tf.InteractiveSession()
sess.run(init)
#learning_rate_ = [i*0.001 for i in range(20,10,-1)]
#for learning_rate in [0.02, 0.01]:
#    feed = {lr:learning_rate, BatchSize: batch_size}
N = int(len(valdata.Y) / batch_size) + 1
for i in range(training_epochs):
    feed = {lr:learning_rate, BatchSize: batch_size}
    
    for n in range(N):
        x,y = traindata.getBatchData()
        feed[X], feed[Y] = x, y
        step_loss_prev = step_loss
        _, step_loss = sess.run([train, cost], feed_dict=feed)
        cost_history = np.append(cost_history,step_loss)
        
    y_pred = sess.run(tf.argmax(Y_pred,1),feed_dict={X: valdata.X, BatchSize: len(valdata.Y)})
    y_true = sess.run(tf.argmax(valdata.Y,1))
    accuracy_val = accuracy_score(y_pred, y_true)
    
    print("[step: {}] loss: {}".format(i, step_loss), "\tvalidation: {:.3f}%".format(accuracy_val * 100))    
    
print('')
saver.save(sess, model_path)
sess.close()

[step: 0] loss: 0.6642764806747437 	validation: 72.327%
[step: 1] loss: 0.4816092848777771 	validation: 78.460%
[step: 2] loss: 0.387756884098053 	validation: 82.819%
[step: 3] loss: 0.3641592264175415 	validation: 84.236%
[step: 4] loss: 0.3416106104850769 	validation: 85.018%
[step: 5] loss: 0.3418843746185303 	validation: 85.184%
[step: 6] loss: 0.31423819065093994 	validation: 86.011%
[step: 7] loss: 0.30728307366371155 	validation: 86.016%
[step: 8] loss: 0.27657267451286316 	validation: 86.739%
[step: 9] loss: 0.2866363525390625 	validation: 88.737%
[step: 10] loss: 0.282417356967926 	validation: 89.229%
[step: 11] loss: 0.2790994346141815 	validation: 89.853%
[step: 12] loss: 0.2722412943840027 	validation: 89.628%
[step: 13] loss: 0.27140945196151733 	validation: 89.608%
[step: 14] loss: 0.2405080795288086 	validation: 90.066%
[step: 15] loss: 0.26035743951797485 	validation: 89.927%
[step: 16] loss: 0.2578655481338501 	validation: 90.043%
[step: 17] loss: 0.2742122411727905 	v

[step: 142] loss: 0.16030333936214447 	validation: 93.948%
[step: 143] loss: 0.13213954865932465 	validation: 94.088%
[step: 144] loss: 0.14252784848213196 	validation: 93.905%
[step: 145] loss: 0.16209042072296143 	validation: 93.773%
[step: 146] loss: 0.1557134985923767 	validation: 94.111%
[step: 147] loss: 0.14099454879760742 	validation: 94.263%
[step: 148] loss: 0.12422593683004379 	validation: 94.061%
[step: 149] loss: 0.16248267889022827 	validation: 93.925%
[step: 150] loss: 0.14798504114151 	validation: 94.236%
[step: 151] loss: 0.1216229498386383 	validation: 93.980%
[step: 152] loss: 0.15735793113708496 	validation: 94.077%
[step: 153] loss: 0.15195557475090027 	validation: 94.204%
[step: 154] loss: 0.15862473845481873 	validation: 93.952%
[step: 155] loss: 0.13840103149414062 	validation: 94.002%
[step: 156] loss: 0.17088769376277924 	validation: 93.859%
[step: 157] loss: 0.13311801850795746 	validation: 93.998%
[step: 158] loss: 0.13870546221733093 	validation: 94.243%
[s

In [ ]:
sess = tf.InteractiveSession()
saver.restore(sess, model_path)
y_pred = sess.run(tf.argmax(Y_pred,1),feed_dict={X: testdata.X, BatchSize: len(testdata.Y)})
y_true = sess.run(tf.argmax(testdata.Y,1))
print(y_pred.shape, y_true.shape)

In [ ]:
sess.close()

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

fig = plt.figure(figsize=(10,8))
plt.plot(cost_history)
plt.ylabel("Cost")
plt.xlabel("Iterations") 
plt.axis([0,len(cost_history),0,np.max(cost_history)])
plt.show()


p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
print("F-Score:", round(f,3))
print("Accuracy: ", accuracy_score(y_true, y_pred))

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

In [ ]:
'''
model_path_f = '../models/RNN/'
filename = 'my_RNN_model_S9_40.meta'


sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
loader = tf.train.import_meta_graph(model_path_f+filename)
loader.restore(sess, tf.train.latest_checkpoint(model_path_f))

SR = 22050
####
justone = True

while(justone):
    justone = False
    #print("start to record the audio.")
    
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    #print("Recording finished.")
    stream.stop_stream()
    stream.close()

    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    ####
    filename1 = '../data/phantom/JUNE_01_PHANTOMS/wavs/22050/WSU_P2_LOADED_BACK_AND_FORTH.wav'
    filename2 = '../data/phantom/JUNE_02_BACKGROUND/wavs/background/canopy_heavy_wind.wav'
    
    sample, sample_rate = librosa.load(filename1,SR)
    print(sample.shape)
    
    
    freqs, times, spectrogram = log_specgram(sample, sample_rate)    
    #showFreqTime([[sample, filename1, SR]])  

    spectrogram = (spectrogram - mean) / std
    
    dataX = spectrogram
    #print(dataX.shape)
    #print('delta shape:',dataX.shape)

    X_hot_list= []
    #print(dataX.shape[0] - seq_length+1)
    for i in range(0, dataX.shape[0] - seq_length+1):
        _x = dataX[i:i + seq_length]
        X_hot_list.append(_x)
    X_hot = np.array(X_hot_list[:])
    #print(X_hot[0])
    #print('\n\n\n')
    y_pred = sess.run(Y_pred,feed_dict={X: X_hot})
    #y_pred[y_pred<0.5] = 0
    #y_pred[y_pred>=0.5] = 1
    print(y_pred[20:30] )
    y_true = np.ones(shape=[y_pred.shape[0]])
    y_pred[y_pred<0.5] = 0
    y_pred[y_pred>=0.5] = 1
    
    p,r,f,s = precision_recall_fscore_support(y_true, y_pred, average='micro')
    print("F-Score:", round(f,3))
    print("Accuracy: ", accuracy_score(y_true, y_pred))

    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))

    
    if y_pred[0] == 1:
        print('The sound is Drone')
    else :
        print('THe sound isn\'t Drone')
    

sess.close()
'''